## 实战tensorflow2.0 搭建一个ANN，全连接神经网络

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import functools

In [2]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"
train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TRAIN_DATA_URL)

In [3]:
np.set_printoptions(precision=3, suppress=True)  ## 控制文件中小数点的位数
## np.set_printoptions(precision=3, suppress=True)
# 参数：
# precision: 保留几位小数，后面不会补0
# supress: 对很大/小的数不使用科学计数法 (true)
# formatter: 强制格式化，后面会补0

In [4]:
type(train_file_path)

str

In [5]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [6]:
def get_dataset(file_path, **kwargs):
    dataset = tf.data.experimental.make_csv_dataset(
    file_path,
    batch_size=5,
    label_name=LABEL_COLUMN,
    na_value="?",
    num_epochs=1,
    ignore_errors=True,
    **kwargs)
    return dataset

In [7]:
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [8]:
def show_batch(dataset):
    for batch, label in dataset.take(1):
        for key, value in batch.items():
            print("{:20s}:{}".format(key, value.numpy()))

In [9]:
show_batch(raw_train_data)

sex                 :[b'female' b'male' b'male' b'male' b'male']
age                 :[34. 40. 80. 22. 25.]
n_siblings_spouses  :[0 0 0 0 1]
parch               :[0 0 0 0 0]
fare                :[13.    27.721 30.     7.125  7.775]
class               :[b'Second' b'First' b'First' b'Third' b'Third']
deck                :[b'unknown' b'unknown' b'A' b'unknown' b'unknown']
embark_town         :[b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               :[b'y' b'y' b'y' b'y' b'n']


In [10]:
## 如果文件第一行没有列名，可以手动加列名
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']
temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)
show_batch(temp_dataset)

sex                 :[b'male' b'male' b'male' b'female' b'male']
age                 :[30.  4. 41. 24. 28.]
n_siblings_spouses  :[0 4 2 0 0]
parch               :[0 1 0 2 0]
fare                :[ 7.896 29.125 14.108 14.5    7.896]
class               :[b'Third' b'Third' b'Third' b'Second' b'Third']
deck                :[b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         :[b'Southampton' b'Queenstown' b'Southampton' b'Southampton' b'Southampton']
alone               :[b'y' b'n' b'n' b'n' b'y']


In [11]:
## select_columns 选中固定的列
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)
show_batch(temp_dataset)

age                 :[18. 28. 38. 71. 34.]
n_siblings_spouses  :[0 0 0 0 1]
class               :[b'Third' b'Third' b'First' b'First' b'Second']
deck                :[b'unknown' b'unknown' b'C' b'A' b'unknown']
alone               :[b'n' b'y' b'n' b'y' b'n']


In [12]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0]
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS, column_defaults=DEFAULTS)
show_batch(temp_dataset)

age                 :[31.    0.83 28.   52.    5.  ]
n_siblings_spouses  :[1. 0. 1. 0. 0.]
parch               :[0. 2. 0. 0. 0.]
fare                :[52.    29.    16.1   30.5   12.475]


In [13]:
example_batch, labels_batch = next(iter(temp_dataset)) 

## Data preprocessing

## 常用的几种数据预处理方式
### 1、sklearn+pandas+numpy
### 2、tensorflow内置处理接口
### 在模型内部进行预处理的主要优点是，在导出模型时，它包括预处理。 这样，您可以将原始数据直接传递到模型。

In [ ]:
# x = tf.constant([1, 4])
# y = tf.constant([2, 5])
# z = tf.constant([3, 6])
# tf.stack([x, y, z])

In [14]:
def pack(features, label):
    return tf.stack(list(features.values()), axis=-1), label

In [15]:
packed_dataset = temp_dataset.map(pack)
for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

[[28.    0.    0.   33.  ]
 [33.    1.    2.   27.75]
 [45.    0.    0.   35.5 ]
 [28.    0.    0.    6.95]
 [28.    0.    0.   79.2 ]]

[1 1 0 0 1]


In [16]:
example_batch, labels_batch = next(iter(temp_dataset))

In [17]:
class PackNumericFeatures(object):
    def __init__(self, names):
        self.names = names
    
    def __call__(self, features, labels):
        numeric_features = [features.pop(name) for name in self.names]
        numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
        numeric_features = tf.stack(numeric_features, axis=-1)
        features['numeric'] = numeric_features
        
        return features, labels

In [18]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch', 'fare']
packed_train_data = raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
paced_test_data = raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [19]:
show_batch(packed_train_data)

sex                 :[b'female' b'female' b'female' b'male' b'female']
class               :[b'First' b'First' b'First' b'Third' b'Third']
deck                :[b'D' b'unknown' b'B' b'unknown' b'unknown']
embark_town         :[b'Southampton' b'Cherbourg' b'Southampton' b'Southampton' b'Southampton']
alone               :[b'n' b'y' b'n' b'y' b'n']
numeric             :[[ 63.      1.      0.     77.958]
 [ 28.      0.      0.    110.883]
 [ 17.      1.      0.     57.   ]
 [ 16.      0.      0.      9.217]
 [ 40.      1.      0.      9.475]]


In [27]:
example_batch, labels_batch = next(iter(packed_train_data)) 

## Data Normalization

In [20]:
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [21]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [22]:
def normalize_numeric_data(data, mean, std):
    return(data - mean) / std

In [23]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x0000019233DA7488>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [28]:
example_batch['numeric']

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  8.   ,  2.   , 69.55 ],
       [25.   ,  0.   ,  0.   ,  7.65 ],
       [48.   ,  1.   ,  0.   , 52.   ],
       [ 6.   ,  0.   ,  1.   , 12.475],
       [16.   ,  0.   ,  1.   , 39.4  ]], dtype=float32)>

In [29]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 ,  6.476,  2.043,  0.644],
       [-0.37 , -0.474, -0.479, -0.49 ],
       [ 1.468,  0.395, -0.479,  0.323],
       [-1.889, -0.474,  0.782, -0.401],
       [-1.089, -0.474,  0.782,  0.092]], dtype=float32)

## Categorical data
### Use the tf.feature_column API to create a collection with a tf.feature_column.indicator_column for each categorical column.

In [30]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [31]:
tf.feature_column.indicator_column??

In [32]:
categorical_columns = []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [33]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

[0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


## Combined preprocessing layer

In [34]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [35]:
print(preprocessing_layer(example_batch).numpy()[0])

[ 0.     1.     0.     0.     1.     0.     0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.    -0.13   6.476
  2.043  0.644  1.     0.   ]


## Build the model

In [38]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1),
                          
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             optimizer='adam',
             metrics=['accuracy'])

In [39]:
train_data = packed_train_data.shuffle(500)
test_data = paced_test_data

In [40]:
model.fit(train_data, epochs=20)

Epoch 1/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4990 - accuracy: 0.7416
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4144 - accuracy: 0.8214
Epoch 3/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4072 - accuracy: 0.8182
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3955 - accuracy: 0.8389
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3828 - accuracy: 0.8293
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3747 - accuracy: 0.8373
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3693 - accuracy: 0.8325
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3627 - accuracy: 0.8469
Epoch 9/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3598 - accuracy: 0.8325
Epoch 10/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3498 - accuracy: 0.8469

In [41]:
test_loss, test_accuracy = model.evaluate(test_data)
print('\n\nTest Loss {}, Test Accuracy {}'.format(test_loss, test_accuracy))

126/126 [==============================] - 0s 3ms/step - loss: 0.2881 - accuracy: 0.8692


Test Loss 0.288139671087265, Test Accuracy 0.8692185282707214


In [42]:
# !pip install mlxtend

In [43]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix